In [1]:
import re
import pandas as pd
import numpy as np
from collections import defaultdict
from nltk import tokenize


from nltk.sentiment.vader import SentimentIntensityAnalyzer

import matplotlib.pyplot as plt
import matplotlib
plt.style.use('fivethirtyeight')
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from tqdm import tqdm_notebook as tqdm
from tqdm import trange


import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re


pd.set_option('display.max_rows', 500)


def getcorpus_local(local):
    with open(str(local)) as f:
        soup = BeautifulSoup(f, "html.parser", exclude_encodings=["iso-8859-7"])
        words = soup.get_text()
        #words= re.sub("[^a-zA-Z' ]+", " ", words)
        words = re.sub("\\n","",words)
        words =re.sub("\\'", "", words)
        return words
    
HPOne = getcorpus_local(R"C:\Users\heather\Desktop\Post Kaiser Projects\Harry Potter\Harry Potter and the Sorcerer.TXT")

HPOne = HPOne[3:]

In [2]:
##############################################################################################################################
#
#
#                                           Import all the things!
#
#
##############################################################################################################################


import tensorflow as tf
import numpy as np
import os
import time

In [3]:
##############################################################################################################################
#
#
#                                           Get data
#
#
##############################################################################################################################





path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


In [4]:
##############################################################################################################################
#
#
#                                           Process text
#
#
##############################################################################################################################




# Read, then decode for py2 compat.
#text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
#text =open(r"C:\Users\heather\Desktop\Post Kaiser Projects\Harry Potter\Harry Potter and the Sorcerer.txt", 'rb').read().strip()
text = HPOne


# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 433563 characters


In [5]:
print(text[:250])

Harry Potter and the Sorcerers Stone CHAPTER ONE THE BOY WHO LIVED Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people youd expect to be involved in any


In [6]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

80 unique characters


In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])


print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ' ' :   0,
  '!' :   1,
  '"' :   2,
  '(' :   3,
  ')' :   4,
  '*' :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '0' :   9,
  '1' :  10,
  '2' :  11,
  '3' :  12,
  '4' :  13,
  '5' :  14,
  '6' :  15,
  '7' :  16,
  '8' :  17,
  '9' :  18,
  ':' :  19,
  ...
}


In [8]:
# Show how the first 13 characters from the text are mapped to integers
print('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'Harry Potter ' ---- characters mapped to int ---- > [29 49 66 66 73  0 37 63 68 68 53 66  0]


In [9]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

H
a
r
r
y


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Harry Potter and the Sorcerers Stone CHAPTER ONE THE BOY WHO LIVED Mr. and Mrs. Dursley, of number fo'
'ur, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were t'
'he last people youd expect to be involved in anything strange or mysterious, because they just didnt '
'hold with such nonsense. Mr. Dursley was the director of a firm called Grunnings, which made drills. '
'He was a big, beefy man with hardly any neck, although he did have a very large mustache. Mrs. Dursle'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Harry Potter and the Sorcerers Stone CHAPTER ONE THE BOY WHO LIVED Mr. and Mrs. Dursley, of number f'
Target data: 'arry Potter and the Sorcerers Stone CHAPTER ONE THE BOY WHO LIVED Mr. and Mrs. Dursley, of number fo'


In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 29 ('H')
  expected output: 49 ('a')
Step    1
  input: 49 ('a')
  expected output: 66 ('r')
Step    2
  input: 66 ('r')
  expected output: 66 ('r')
Step    3
  input: 66 ('r')
  expected output: 73 ('y')
Step    4
  input: 73 ('y')
  expected output: 0 (' ')


In [14]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [15]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [17]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 80) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           20480     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 80)            82000     
Total params: 4,040,784
Trainable params: 4,040,784
Non-trainable params: 0
_________________________________________________________________


In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
sampled_indices

array([18, 70, 28, 26, 55, 30,  9,  6, 76, 73,  5,  4, 37, 19, 58, 58, 11,
       26, 53,  8, 39, 52, 52, 42, 66, 50, 20, 36, 35, 43, 69,  2,  4,  6,
       33, 15, 29, 22, 43,  7, 21, 60, 73, 68, 27, 49, 54, 47,  4, 32, 73,
       70,  8, 65, 70, 10, 57, 34, 46, 22, 39, 12, 22, 66, 67, 48, 55, 13,
       59, 72, 74, 66,  9, 57, 67, 48, 55, 50, 70, 70, 46, 46, 57, 73, 35,
        0, 21,  2, 48, 30, 28,  4, 25, 36, 43, 23, 46, 67, 77, 23],
      dtype=int64)

In [22]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ad in his whole life -- more money than even Dudley had ever had. "Might as well get yer uniform," s'

Next Char Predictions: 
 '9vGEgI0,ây*)P:jj2Ee.RddUrb;ONVu"),L6HAV-?lytFafZ)Kyv.qv1iMYAR3Ars\\g4kxzr0is\\gbvvYYiyN ?"\\IG)DOVBYsœB'


In [23]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 80)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.382399


In [24]:
model.compile(optimizer='adam', loss=loss)

In [25]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [26]:
EPOCHS = 100

In [27]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
67/67 [==============================] - 139s 2s/step - loss: 3.2449
Epoch 2/100
67/67 [==============================] - 136s 2s/step - loss: 2.3141
Epoch 3/100
67/67 [==============================] - 134s 2s/step - loss: 2.0992
Epoch 4/100
67/67 [==============================] - 134s 2s/step - loss: 1.9131
Epoch 5/100
67/67 [==============================] - 134s 2s/step - loss: 1.7595
Epoch 6/100
67/67 [==============================] - 147s 2s/step - loss: 1.6320
Epoch 7/100
67/67 [==============================] - 145s 2s/step - loss: 1.5288
Epoch 8/100
67/67 [==============================] - 151s 2s/step - loss: 1.4481
Epoch 9/100
67/67 [==============================] - 145s 2s/step - loss: 1.3799
Epoch 10/100
67/67 [==============================] - 150s 2s/step - loss: 1.3239
Epoch 11/100
67/67 [==============================] - 137s 2s/step - loss: 1.2757
Epoch 12/100
67/67 [==============================] - 159s 2s/step - loss: 1.2323
Epoch 13/100
67/67 [=====

In [28]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_100'

In [29]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            20480     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 80)             82000     
Total params: 4,040,784
Trainable params: 4,040,784
Non-trainable params: 0
_________________________________________________________________


In [31]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [35]:
print(generate_text(model, start_string=u"owl "))

owl to die." "Th the best team were cheering. What they were rest of the first years. Even Neville thought of Hagrid, expelled but peture was a small voice in a low, with relatives -- George get to Professor Dumbledore and Hermione read the cuptofth matter if yeh know wouldnt be sure I dont know..." said Ron finally. "Flamels juct go somewhe points from Gryffindor." "Hes just made that rule up," Harry muttered an rup a lot -- great with animals." Harry wondered if Hagrid had taken Harry aside. "Dont put- on Saturday," said Ron. "Just come proveled into a gige, but Snape tried to robsame of such a telephine pain in his truth concerent wild eyes on all the times Dudley had punched him on the very last thing they werent surprised straper little reasons were delivered to Harry. He looked voice. "Then Im going to do magic." Harry croaked, "that was what happened when Vol-, sorry -- I mean, the name of just be offered for a Snitch teams and something reflected because his ron. Harry walked m

In [36]:
print(generate_text(model, start_string=u"blue "))

blue and hat much lo;er, too -- well -- noble to use them." "Its lucky I got it to kill mention, Dursley and Malfoy darking behind his make a giant called after him to hurry up, I cant breathe!" Harry gave a sloud, but not expection," Harry heard Hermione snap. "Its Wing-gar-dium Levi-o-sa, tas no snapes leston. It took reaching the twins. "Oh, him -- Im werent you, and up in the air behind him. If she was mean an --" Quirrell stood up. "So -- about that the exams werent ride packed with Muggles?" "If thats water in my poss Snatch. Binns had gone. He flew in cabe hed rather r to block the unicorns -- I could have been crossbow, and all the way up the street that someone got to go andward, but it was still looking forward to the house. He was still determined not to meet his family and sad been co they shouted, as though this was as horrible thought to look in front. reat, who had been wanting. The way to his forehead with an orner the high chair. Non he had perfoy to come bursting thro